## Hypothesis:
Violent crime could be affected by the weather conditions (e.g. more homicides when more people are outside in nice weather)

## Problem:
Predict the number of violent crimes in an urban area based on weather conditions

## Data:
* Chicago crime data from beginning of 2016 to present
* Historic weather data for Chicago for every day/hour from Crime dataset

In [1]:
import json  #handling API requests
import csv  #used when downloading Csvs
import pandas as pd  #main python module for data cleaning and analysis
import numpy as np  #for use with sklearn
import datetime  #working with timestamps from API
from datetime import timedelta, date  #more granular imports
import time  # used to time functions
import requests  # Main Python module for making HTTP Requests
import os  #used when Python needs to work with the OS/machine it's being processed on 
from pandas.io.json import json_normalize
import datetime

In [69]:
put_file(credentials_2,'joined_data.csv')

<Response [201]>


In [2]:
from io import StringIO
import requests
import json
import pandas as pd

# @hidden_cell
# This function accesses a file in your Object Storage. The definition contains your credentials.
# You might want to remove those credentials before you share your notebook.
def get_object_storage_file_with_credentials_f2807a6a3891460882043342cc4b184b(container, filename):
    """This functions returns a StringIO object containing
    the file content from Bluemix Object Storage."""

    url1 = ''.join(['https://identity.open.softlayer.com', '/v3/auth/tokens'])
    data = {'auth': {'identity': {'methods': ['password'],
            'password': {'user': {'name': 'member_5a1b8103fd33ca58816f013a4ab0a5bc51ec39e8','domain': {'id': '6620e3eb4b434453b2702fe8b96f7a2e'},
            'password': 'Dz54IeQKe7r.E!as'}}}}}
    headers1 = {'Content-Type': 'application/json'}
    resp1 = requests.post(url=url1, data=json.dumps(data), headers=headers1)
    resp1_body = resp1.json()
    for e1 in resp1_body['token']['catalog']:
        if(e1['type']=='object-store'):
            for e2 in e1['endpoints']:
                        if(e2['interface']=='public'and e2['region']=='dallas'):
                            url2 = ''.join([e2['url'],'/', container, '/', filename])
    s_subject_token = resp1.headers['x-subject-token']
    headers2 = {'X-Auth-Token': s_subject_token, 'accept': 'application/json'}
    resp2 = requests.get(url=url2, headers=headers2)
    return StringIO(resp2.text)

chicago = pd.read_csv(get_object_storage_file_with_credentials_f2807a6a3891460882043342cc4b184b('ChicagoCrimeData', 'Crimes_-_2016_to_present.csv'))
chicago.head()


,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,10508593,HZ250410,05/03/2016 09:00:00 PM,042XX W MADISON ST,1811,NARCOTICS,POSS: CANNABIS 30GMS OR LESS,SIDEWALK,True,False,...,28,26,18,1148220.0,1899677.0,2016,05/10/2016 03:56:50 PM,41.880658,-87.731212,"(41.880658176, -87.731212135)"
1,10508594,HZ250385,05/03/2016 09:35:00 PM,065XX S KEDVALE AVE,0810,THEFT,OVER $500,APARTMENT,False,True,...,13,65,06,1149830.0,1860872.0,2016,05/10/2016 03:56:50 PM,41.774141,-87.726307,"(41.77414081, -87.726307093)"
2,10508595,HZ250362,05/03/2016 02:45:00 PM,045XX S DAMEN AVE,0560,ASSAULT,SIMPLE,STREET,False,True,...,12,61,08A,1163662.0,1874660.0,2016,05/10/2016 03:56:50 PM,41.811698,-87.675214,"(41.811697721, -87.675214316)"
3,10508597,HZ250202,05/03/2016 06:42:00 PM,063XX S CLAREMONT AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,SIDEWALK,False,True,...,15,66,08B,1161829.0,1862607.0,2016,05/10/2016 03:56:50 PM,41.778661,-87.682273,"(41.778661058, -87.68227253)"
4,10508598,HZ250298,05/03/2016 03:00:00 PM,070XX S INDIANA AVE,0320,ROBBERY,STRONGARM - NO WEAPON,SIDEWALK,False,False,...,6,69,03,1178808.0,1858478.0,2016,05/10/2016 03:56:50 PM,41.766961,-87.620152,"(41.766961034, -87.620152116)"


In [4]:
include_values = ['ASSAULT'
,'BATTERY'
,'ROBBERY'
,'SEX OFFENSE'
,'HOMICIDE'
,'CRIM SEXUAL ASSAULT'
,'KIDNAPPING'
,'ARSON'
,'HUMAN TRAFFICKING']

In [5]:
chicago.rename(columns=lambda x: x.replace(" ", '_'), inplace=True)

In [6]:
chicago.rename(columns=lambda x: x.lower(), inplace=True)

In [7]:
chicago['time'] = chicago.date.apply(lambda x: x[12:])

In [8]:
chicago.date = pd.to_datetime(chicago.date)

In [9]:
chicago['hour'] = chicago.date.apply(lambda x: x.hour)

In [10]:
chicago['time'], chicago.date = chicago.date.apply(lambda x: x.time()), chicago.date.apply(lambda x: x.date())

In [11]:
def getDates(start_date, end_date, delta = 30):
    '''
    Returns a list of date intervals of delta number of days, that encompasses the entire duration between start_date and end_date
    start_date: starting date in format YYYYMMDD
    end_date: end date in format YYYYMMDD
    delta: set as number of days for the interval; the interval between start_date and end_date; default = 30
    '''
    from datetime import timedelta, date
    delta = delta -1
    #Convert dates to date format
    s = datetime.date(int(str(start_date)[:4]),int(str(start_date)[4:6]),int(str(start_date)[6:]))
    e = datetime.date(int(str(end_date)[:4]),int(str(end_date)[4:6]),int(str(end_date)[6:]))

    #Set delta
    if (int(delta) > 30 or int(delta) < 1):
        raise ValueError('Error: delta out of range. 1 <= delta <= 30')
    else:
        delta = timedelta(days=int(delta))

    days= []
    day_from = s
    while (day_from) < e:
        day_to = day_from + delta
        days.append((day_from,day_to))
        print('...for range %s to %s ...' % (day_from.strftime("%Y%m%d"), day_to.strftime("%Y%m%d")))
        day_from += delta
    return days

In [12]:
dates = getDates(20160101, 20170317)

...for range 20160101 to 20160130 ...
...for range 20160130 to 20160228 ...
...for range 20160228 to 20160328 ...
...for range 20160328 to 20160426 ...
...for range 20160426 to 20160525 ...
...for range 20160525 to 20160623 ...
...for range 20160623 to 20160722 ...
...for range 20160722 to 20160820 ...
...for range 20160820 to 20160918 ...
...for range 20160918 to 20161017 ...
...for range 20161017 to 20161115 ...
...for range 20161115 to 20161214 ...
...for range 20161214 to 20170112 ...
...for range 20170112 to 20170210 ...
...for range 20170210 to 20170311 ...
...for range 20170311 to 20170409 ...


In [13]:
chicago_mod = chicago[chicago['primary_type'].isin(include_values)].reset_index(drop=True)

In [14]:
include_columns = ['date', 'hour', 'primary_type','district']

In [15]:
chicago_final = chicago_mod.filter(include_columns)

In [16]:
gb = chicago_final.sort_values(by=['date'])

In [19]:
lat = 41.811698
lon = -87.675214
units  = 'm'
startDate = dates[0][0].strftime('%Y%m%d')
endDate = dates[0][1].strftime('%Y%m%d')

In [20]:
url = "http://api.weather.com/v1/geocode/" + str(lat) + "/" + str(lon)+ \
    "/observations/historical.json?apiKey=" + my_apikey + \
    "&units=" + units + \
    "&startDate="+str(startDate) + \
    "&endDate="+str(endDate)

In [21]:
r = requests.get(url)

In [22]:
r

<Response [200]>

In [23]:
#listFrames = []
weather = pd.DataFrame()
for i in dates:
    startDate = i[0].strftime('%Y%m%d')
    endDate = i[1].strftime('%Y%m%d')
    #print(startDate,endDate)
    url = "http://api.weather.com/v1/geocode/" + str(lat) + "/" + str(lon)+ \
        "/observations/historical.json?apiKey=" + my_apikey + \
        "&units=" + units + \
        "&startDate="+str(startDate) + \
        "&endDate="+str(endDate)
    r = requests.get(url).json()
    wide_format = json_normalize(r['observations'])
    #listFrames.append(wide_format)
    weather = pd.concat((weather, wide_format), axis = 0)
    time.sleep(7)

In [24]:
weather.head()

,blunt_phrase,class,clds,day_ind,dewPt,expire_time_gmt,feels_like,gust,heat_index,icon_extd,...,uv_index,valid_time_gmt,vis,water_temp,wc,wdir,wdir_cardinal,wspd,wx_icon,wx_phrase
0,None,observation,BKN,N,-9,1451638380,-11.0,NaN,-4,2700,...,0,1451631180,14.0,None,-11.0,250.0,WSW,22.0,27,Mostly Cloudy
1,None,observation,SCT,N,-11,1451639460,-12.0,46.0,-4,2900,...,0,1451632260,14.0,None,-12.0,260.0,W,30.0,29,Partly Cloudy
2,None,observation,FEW,N,-11,1451641980,-13.0,NaN,-5,3300,...,0,1451634780,14.0,None,-13.0,250.0,WSW,30.0,33,Fair
3,None,observation,FEW,N,-11,1451645580,-14.0,NaN,-6,3300,...,0,1451638380,14.0,None,-14.0,260.0,W,30.0,33,Fair
4,None,observation,SCT,N,-12,1451649180,-13.0,NaN,-6,2900,...,0,1451641980,14.0,None,-13.0,260.0,W,20.0,29,Partly Cloudy


In [25]:
weather['obs_time'] = weather.valid_time_gmt.apply(lambda x: datetime.datetime.fromtimestamp(x) - datetime.timedelta(hours = 6))

In [26]:
weather['obs_date'] , weather['obs_hour']= weather.obs_time.apply(lambda x: x.date()), weather.obs_time.apply(lambda x: x.hour)

In [28]:
grouped = chicago_final.groupby(['date', 'hour', 'district']).count()

In [29]:
grouped = grouped.reset_index()

In [30]:
grouped.rename(columns=lambda x: x.replace('primary_type', 'crime_counts'), inplace=True)

In [31]:
weather_cols = ['obs_date','obs_hour','heat_index', 'precip_hrly']

In [32]:
weather_final = weather.filter(weather_cols)

In [34]:
weather_final.rename(columns= lambda x: x.replace('obs_', ''), inplace = True)

In [35]:
joined = pd.merge(grouped, weather_final,  how='inner', left_on=['date','hour'], right_on = ['date', 'hour'])

In [38]:
joined.to_csv('joined_data.csv')

In [67]:
from io import BytesIO  
import requests  
import json  
import pandas as pd

def put_file(credentials, local_file_name):  
    """This functions returns a StringIO object containing
    the file content from Bluemix Object Storage V3."""
    f = open(local_file_name,'r')
    my_data = f.read()
    url1 = ''.join(['https://identity.open.softlayer.com', '/v3/auth/tokens'])
    data = {'auth': {'identity': {'methods': ['password'],
            'password': {'user': {'name': credentials['username'],'domain': {'id': credentials['domain_id']},
            'password': credentials['password']}}}}}
    headers1 = {'Content-Type': 'application/json'}
    resp1 = requests.post(url=url1, data=json.dumps(data), headers=headers1)
    resp1_body = resp1.json()
    for e1 in resp1_body['token']['catalog']:
        if(e1['type']=='object-store'):
            for e2 in e1['endpoints']:
                        if(e2['interface']=='public'and e2['region']=='dallas'):
                            url2 = ''.join([e2['url'],'/', credentials['container'], '/', local_file_name])
    s_subject_token = resp1.headers['x-subject-token']
    headers2 = {'X-Auth-Token': s_subject_token, 'accept': 'application/json'}
    resp2 = requests.put(url=url2, headers=headers2, data = my_data )
    print(resp2)

## Modelling
Linear regression to predict number of crimes in the area

First model used police district dummies with weather data

In [41]:
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import train_test_split

In [73]:
final_data = pd.get_dummies(joined, columns = ['district'], drop_first=True)

y = final_data.crime_counts
X = final_data.drop(['crime_counts', 'date', 'hour'], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 100)

linreg = LinearRegression()

model = linreg.fit(X_train, y_train)

predictions = model.predict(X_test)

model.score(X_test, y_test)

0.02516381957771574

Model 2: dropped police district info in hopes to increase R^2. It did not help.

In [74]:
y2 = joined['crime_counts']

In [80]:
X2 = joined.drop(['crime_counts', 'date', 'hour', 'district'], axis = 1)

In [81]:
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X2, y2, test_size = 0.25, random_state = 100)

In [82]:
linreg2 = LinearRegression()
model2 = linreg2.fit(X_train_2, y_train_2)


In [83]:
predictions2 = model2.predict(X_test_2)

In [84]:
model2.score(X_test_2, y_test_2)

0.0019744613825671831

In [85]:
model.score?

Model 3: no districts, average crime_counts by hour.

In [104]:
data_3 = joined.drop('district', axis = 1)

In [94]:
data_3 = data_3.groupby(['date', 'hour']).mean()

In [200]:
data_3 = data_3.reset_index()

In [97]:
y3 = data_3.crime_counts
X3 = data_3.drop(['crime_counts'], axis = 1)

In [98]:
X_train3, X_test3, y_train3, y_test3 = train_test_split(X3, y3, test_size = 0.25, random_state = 100)

In [99]:
linreg3 = LinearRegression()
model3 = linreg3.fit(X_train3, y_train3)

In [101]:
pred3 = model3.predict(X_test3)

In [102]:
model3.score(X_test3, y_test3)

0.0068737833645068367

Model 4: not counting date and time

In [105]:
y4 = data_3.crime_counts
X4 = data_3.drop(['crime_counts', 'date', 'hour'], axis = 1)
X_train4, X_test4, y_train4, y_test4 = train_test_split(X4, y4, test_size = 0.25, random_state = 100)

In [106]:
linreg4 = LinearRegression()
model4 = linreg4.fit(X_train4, y_train4)

In [107]:
pred4 = model4.predict(X_test4)

In [108]:
model4.score(X_test4, y_test4)

0.0019744613825671831

Aggregated absolute number of crimes by day, average heat index, average hourly precipitation

In [158]:
data_5 = joined.drop('district', axis = 1)

In [159]:
data_5 = data_5.groupby('date').sum()

In [160]:
data_5 = data_5.reset_index()

In [161]:
data_5 = data_5.drop(['hour', 'heat_index'], axis = 1)

In [162]:
data_5['heat_index'] = data_4['heat_index']
data_5['precip_hrly'] = data_4['precip_hrly']

In [164]:
y6 = data_5.crime_counts
X6 = data_5.drop(['crime_counts','date'], axis = 1)

In [165]:
X_train6, X_test6, y_train6, y_test6 = train_test_split(X6, y6, test_size = 0.25, random_state = 100)

In [166]:
linreg6 = LinearRegression()


In [167]:
model6 = linreg6.fit(X_train6, y_train6)

In [168]:
pred6 = model6.predict(X_test6)

In [169]:
model6.score(X_test6, y_test6)

0.24687111248490679

## Best result so far: 
### Aggregated average number of crimes by day

In [109]:
data_4 = joined.drop('district', axis = 1)

In [111]:
data_4 = data_4.groupby('date').mean()

In [114]:
data_4 = data_4.drop('hour', axis = 1)

In [116]:
data_4 = data_4.reset_index()

In [124]:
y5 = data_4.crime_counts
X5 = data_4.drop(['crime_counts','date'], axis = 1)

In [125]:
X_train5, X_test5, y_train5, y_test5 = train_test_split(X5, y5, test_size = 0.25, random_state = 100)

In [126]:
linreg5 = LinearRegression()


In [127]:
model5 = linreg5.fit(X_train5, y_train5)

In [128]:
pred5 = model5.predict(X_test5)

In [129]:
model5.score(X_test5, y_test5)

0.29826134913082802

## Using more crimes that could be affected by weather

In [170]:
chicago.primary_type.unique()

array(['NARCOTICS', 'THEFT', 'ASSAULT', 'BATTERY', 'ROBBERY',
       'OTHER OFFENSE', 'CRIMINAL DAMAGE', 'DECEPTIVE PRACTICE',
       'LIQUOR LAW VIOLATION', 'BURGLARY', 'WEAPONS VIOLATION',
       'OFFENSE INVOLVING CHILDREN', 'SEX OFFENSE',
       'PUBLIC PEACE VIOLATION', 'MOTOR VEHICLE THEFT',
       'CRIMINAL TRESPASS', 'STALKING', 'CRIM SEXUAL ASSAULT',
       'KIDNAPPING', 'HOMICIDE', 'INTERFERENCE WITH PUBLIC OFFICER',
       'PROSTITUTION', 'GAMBLING', 'INTIMIDATION', 'ARSON', 'OBSCENITY',
       'NON-CRIMINAL', 'PUBLIC INDECENCY', 'HUMAN TRAFFICKING',
       'CONCEALED CARRY LICENSE VIOLATION', 'NON - CRIMINAL',
       'NON-CRIMINAL (SUBJECT SPECIFIED)', 'OTHER NARCOTIC VIOLATION'], dtype=object)

In [180]:
# including more crimes that would depend on weather
chicago2 = pd.read_csv(get_object_storage_file_with_credentials_f2807a6a3891460882043342cc4b184b('ChicagoCrimeData', 'Crimes_-_2016_to_present.csv'))


,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,10508593,HZ250410,05/03/2016 09:00:00 PM,042XX W MADISON ST,1811,NARCOTICS,POSS: CANNABIS 30GMS OR LESS,SIDEWALK,True,False,...,28,26,18,1148220.0,1899677.0,2016,05/10/2016 03:56:50 PM,41.880658,-87.731212,"(41.880658176, -87.731212135)"
1,10508594,HZ250385,05/03/2016 09:35:00 PM,065XX S KEDVALE AVE,0810,THEFT,OVER $500,APARTMENT,False,True,...,13,65,06,1149830.0,1860872.0,2016,05/10/2016 03:56:50 PM,41.774141,-87.726307,"(41.77414081, -87.726307093)"
2,10508595,HZ250362,05/03/2016 02:45:00 PM,045XX S DAMEN AVE,0560,ASSAULT,SIMPLE,STREET,False,True,...,12,61,08A,1163662.0,1874660.0,2016,05/10/2016 03:56:50 PM,41.811698,-87.675214,"(41.811697721, -87.675214316)"
3,10508597,HZ250202,05/03/2016 06:42:00 PM,063XX S CLAREMONT AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,SIDEWALK,False,True,...,15,66,08B,1161829.0,1862607.0,2016,05/10/2016 03:56:50 PM,41.778661,-87.682273,"(41.778661058, -87.68227253)"
4,10508598,HZ250298,05/03/2016 03:00:00 PM,070XX S INDIANA AVE,0320,ROBBERY,STRONGARM - NO WEAPON,SIDEWALK,False,False,...,6,69,03,1178808.0,1858478.0,2016,05/10/2016 03:56:50 PM,41.766961,-87.620152,"(41.766961034, -87.620152116)"


In [181]:
include_values2 = ['ASSAULT'
,'BATTERY'
,'ROBBERY'
,'SEX OFFENSE'
,'HOMICIDE'
,'CRIM SEXUAL ASSAULT'
,'KIDNAPPING'
,'ARSON'
,'HUMAN TRAFFICKING'
,'LIQUOR LAW VIOLATION'
,'PUBLIC PEACE VIOLATION'
,'PUBLIC INDECENCY']

In [184]:
chicago2.rename(columns=lambda x: x.replace(" ", '_'), inplace=True)

In [185]:
chicago2.rename(columns=lambda x: x.lower(), inplace=True)

In [186]:
chicago2['time'] = chicago2.date.apply(lambda x: x[12:])

In [187]:
chicago2.date = pd.to_datetime(chicago2.date)

In [188]:
chicago2['hour'] = chicago2.date.apply(lambda x: x.hour)

In [189]:
chicago2['time'], chicago2.date = chicago2.date.apply(lambda x: x.time()), chicago2.date.apply(lambda x: x.date())

In [190]:
chicago_mod2 = chicago2[chicago2['primary_type'].isin(include_values)].reset_index(drop=True)

In [192]:
include_columns = ['date', 'hour', 'primary_type','district']

In [193]:
chicago_final_2 = chicago_mod2.filter(include_columns)

In [194]:
grouped2 = chicago_final_2.groupby(['date', 'hour', 'district']).count()

In [195]:
grouped2 = grouped2.reset_index()

In [196]:
grouped2.rename(columns=lambda x: x.replace('primary_type', 'crime_counts'), inplace=True)

In [198]:
joined2 = pd.merge(grouped2, weather_final,  how='inner', left_on=['date','hour'], right_on = ['date', 'hour'])